In [1]:
from model.pointpillars import PointPillars

In [2]:
import torch
from torch import nn
from model.pointpillars import PointPillars
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Assuming `model` is the PointPillars model
pretrained_model_path = "/home/linux1/ros2_new_ws/src/ams_motor_drive/scripts/epoch_160.pth"
model_state_dict = torch.load(pretrained_model_path, map_location=torch.device('cpu'))

# Initialize the PointPillars model
model = PointPillars(
    nclasses=3,
    voxel_size=[0.16, 0.16, 4],
    point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
    max_num_points=32,
    max_voxels=(16000, 40000)  # Training and inference max_voxels
)

# Load the filtered state dict into the model (without the head weights)
model.load_state_dict(model_state_dict, strict=False)

# # Now replace the head layer with Identity

model.neck = nn.Identity()
model.head = nn.Identity()

# Set the model to evaluation mode
model.eval()


/tmp/ipykernel_154581/3101015020.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(pretrained_model_path, map_location=torch.device('cpu'))


PointPillars(
  (pillar_layer): PillarLayer(
    (voxel_layer): Voxelization(
      (voxelizer): HardVoxelization()
    )
  )
  (pillar_encoder): PillarEncoder(
    (conv): Conv1d(9, 64, kernel_size=(1,), stride=(1,), bias=False)
    (bn): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  )
  (backbone): Backbone(
    (multi_blocks): ModuleList(
      (0): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (7): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, tra

In [31]:
# Function to load and preprocess PCD file
def load_pcd_file(pcd_path):
    # Load the point cloud data using Open3D
    pcd = o3d.io.read_point_cloud(pcd_path)

    # Convert points to numpy array
    points = np.asarray(pcd.points)  # (N, 3)

    # Check if colors are available and add them
    if pcd.has_colors():
        colors = np.asarray(pcd.colors)  # (N, 3)
        data = np.hstack((points, colors))  # Combine points and colors
    else:
        data = points  # Only points available
        
    return data


# Preprocess the point cloud data for the model
def preprocess_point_cloud(pcd_points):
    # Assuming the model requires specific preprocessing like voxelization
    # Example: Downsample using voxel grid (this is one of many possible approaches)

    # For example, voxel size (you may need to tune this)
    voxel_size = 0.2
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcd_points)
    downsampled_pcd = pcd.voxel_down_sample(voxel_size)

    # Convert the downsampled points to numpy array
    downsampled_points = np.asarray(downsampled_pcd.points)  # Shape: (N', 3)

    # Convert to torch tensor and add batch dimension
    input_tensor = torch.tensor(downsampled_points, dtype=torch.float32).unsqueeze(0)

    return input_tensor


In [32]:
# Inference function
def run_inference(input_tensor):
    with torch.no_grad():
        # Run the model inference (assuming the model returns features directly)
        features = model(input_tensor)

        #for idx, feature in enumerate(features):
            #print(f"Feature {idx}: Type: {type(feature)}, Shape: {feature.shape if isinstance(feature, torch.Tensor) else 'N/A'}")
        
        features = features[-1]      #model provides features on three different scales i.e. at different levels of backbone, last lavel features are more enriched that is why taken. 

    return features

In [33]:
# Example usage
pcd_path = "/home/linux1/Desktop/lidar_01__2023-06-02-21-28-09-468.pcd"
pcd_points = load_pcd_file(pcd_path)

# Preprocess the point cloud data
input_tensor = preprocess_point_cloud(pcd_points)
print(f"input tensor shape = {input_tensor.shape}")

# Run inference
features = run_inference(input_tensor)

B, C, H, W = features.shape  # B=1, C=256, H=62, W=54

# Reshape PointPillars features to [Batch, Seq_Len, Channels]
pointpillars_features_flat = features.permute(0, 2, 3, 1).reshape(B, H * W, C)  # [1, 3348, 256]


# Output the features (can be used for further processing)
print(f"ouput tensor shape = {pointpillars_features_flat.shape}")
print("Extracted Features:", pointpillars_features_flat)

input tensor shape = torch.Size([1, 25829, 3])
ouput tensor shape = torch.Size([1, 3348, 256])
Extracted Features: tensor([[[0.1184, 0.0525, 0.0277,  ..., 0.0801, 0.0000, 0.0000],
         [0.3384, 0.0878, 0.0000,  ..., 0.0000, 0.1475, 0.0000],
         [0.2301, 0.1445, 0.1551,  ..., 0.0000, 0.2247, 0.0000],
         ...,
         [0.1233, 0.1182, 0.2594,  ..., 0.0000, 0.0000, 0.0000],
         [0.1127, 0.1336, 0.2213,  ..., 0.0000, 0.0000, 0.0852],
         [0.3851, 0.0182, 0.1440,  ..., 0.0000, 0.0570, 0.1720]]])
